## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Husavik,IS,66.0449,-17.3389,24.71,84,87,33.49,light snow
1,1,Marathon,US,44.9000,-89.7668,14.25,54,20,1.99,few clouds
2,2,Dikson,RU,73.5069,80.5464,4.48,93,100,18.86,overcast clouds
3,3,Albany,US,42.6001,-73.9662,32.27,79,100,4.74,overcast clouds
4,4,Nenjiang,CN,49.1833,125.2167,-8.03,92,89,4.23,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
11,11,Atuona,PF,-9.8000,-139.0333,77.45,75,22,16.64,few clouds
12,12,Grand Gaube,MU,-20.0064,57.6608,79.20,78,6,1.99,clear sky
15,15,Hilo,US,19.7297,-155.0900,80.33,57,40,0.00,scattered clouds
16,16,Dhidhdhoo,MV,6.8833,73.1000,81.63,70,6,19.75,clear sky
20,20,Butaritari,KI,3.0707,172.7902,81.73,79,92,17.81,light rain
21,21,Avarua,CK,-21.2078,-159.7750,82.45,78,20,13.80,few clouds
27,27,Bambous Virieux,MU,-20.3428,57.7575,79.05,89,75,4.61,light rain
28,28,Puerto Ayora,EC,-0.7393,-90.3518,82.36,76,80,9.51,broken clouds
32,32,Buala,SB,-8.1450,159.5921,79.57,81,93,7.72,overcast clouds
35,35,Hithadhoo,MV,-0.6000,73.0833,80.69,61,27,5.61,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows. method 1
preferred_cities_df.count()

City_ID        179
City           179
Country        178
Lat            179
Lng            179
Max Temp       179
Humidity       179
Cloudiness     179
Wind Speed     179
Description    179
dtype: int64

In [6]:
# 4a. Determine if there are any empty rows. method 2
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        1
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df=preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City_ID        178
City           178
Country        178
Lat            178
Lng            178
Max Temp       178
Humidity       178
Cloudiness     178
Wind Speed     178
Description    178
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
11,Atuona,PF,77.45,few clouds,-9.8000,-139.0333,
12,Grand Gaube,MU,79.20,clear sky,-20.0064,57.6608,
15,Hilo,US,80.33,scattered clouds,19.7297,-155.0900,
16,Dhidhdhoo,MV,81.63,clear sky,6.8833,73.1000,
20,Butaritari,KI,81.73,light rain,3.0707,172.7902,
21,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,
27,Bambous Virieux,MU,79.05,light rain,-20.3428,57.7575,
28,Puerto Ayora,EC,82.36,broken clouds,-0.7393,-90.3518,
32,Buala,SB,79.57,overcast clouds,-8.1450,159.5921,
35,Hithadhoo,MV,80.69,scattered clouds,-0.6000,73.0833,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError): 
        print("No hotel found...skipping.")    
        

No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.
No hotel found...skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name. 
hotel_clean_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
hotel_clean_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
11,Atuona,PF,77.45,few clouds,-9.8000,-139.0333,Villa Enata
12,Grand Gaube,MU,79.20,clear sky,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
15,Hilo,US,80.33,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
16,Dhidhdhoo,MV,81.63,clear sky,6.8833,73.1000,Dhidhoo beach
20,Butaritari,KI,81.73,light rain,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
644,Nhulunbuy,AU,81.21,moderate rain,-12.2333,136.7667,East Arnhem Real Estate
645,Katobu,ID,75.61,overcast clouds,-4.9408,122.5276,La Ode Husein
651,Black River,JM,82.40,few clouds,18.0264,-77.8487,Idlers' Rest Beach Hotel
669,Saint-Joseph,RE,75.56,clear sky,-21.3667,55.6167,"""Plantation Bed and Breakfast"


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))